<a href="https://colab.research.google.com/github/yfan393/CSE6040/blob/main/cse6040_2024_08_27_nb1_efficiency_DRAFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE 6040, Fall 2024: Efficiency (nb1.1.3) [Aug 27]#

Topics in this note:

* Exercise nb1.1.3 — contrasting three methods
* Extension: Sparse dot products

## nb1.1.3 — Decompressing a sparse vector ##

Consider a sparse vector as defined for problem nb1.1.3. A logical vector of length `n`, which is expected to have many zero elements, is stored using a pair of lists that record only the nonzero values and their indices. These two lists are, in turn, wrapped into a dictionary.

In [ ]:
d = {} # sparse
d['vals'] = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
d['inds'] = [0,   3,   7,   3,   3,   5,   1]

This represents the following logical (mathematical) vector:

In [ ]:
#           0    1    2     3
x_true = [1.0, 7.0, 0.0, 11.0, 0.0, 6.0, 0.0, 3.0] # dense representation

In [ ]:
for i, v in zip(range(len(x_true)), x_true): # i, v in enumerate(x_true)
    print(f"Value {v} is at location {i}")

Value 1.0 is at location 0
Value 7.0 is at location 1
Value 0.0 is at location 2
Value 11.0 is at location 3
Value 0.0 is at location 4
Value 6.0 is at location 5
Value 0.0 is at location 6
Value 3.0 is at location 7


In [ ]:
for i in range(len(x_true)):
    print(f"Value {x_true[i]} is at location {i}")

Value 1.0 is at location 0
Value 7.0 is at location 1
Value 0.0 is at location 2
Value 11.0 is at location 3
Value 0.0 is at location 4
Value 6.0 is at location 5
Value 0.0 is at location 6
Value 3.0 is at location 7


Thus, the object `d` is a compact representation of `x_true` when there are many zero components.

Recall that the task in nb1.1.3 is to decompress `d`, producing `x_true`. A natural method is to construct each element of the output, one at a time. For each output, the method scans the sparse vector and accumulates all matching entries.

In [ ]:
for e in zip(d['inds'], d['vals']):
    if e[0] == i: s += e[1]

In [ ]:
for e in zip(...): # is a tuple

In [ ]:
zip([1, 2, 3], [4, 5, 6])

In [ ]:
# Suppose size `d` is `m` (meaning len(d['inds']) and len(d['vals']) are `m`)
def decomp_i(i, d): # calculate `x[i]` from `d`
    s = 0 # O(1)
    for j, v in zip(d['inds'], d['vals']): # execute `m` times => O(m)
        if j == i: # O(1) (constant) independent of `m`
            s += v
    return s # O(1)

In [ ]:
# Demo: Get element 3 from the sparse vector `d`; should print "11.0"
decomp_i(3, d)

11.0

In [ ]:
# Alternative implementation using a list comprehension:
def decomp_i__alt(i, d):
    vals_i = [] ### DEMO: FILL IN ###
    return sum(vals_i)

Below, we use the notation $f(n) = O(n)$ to mean, _informally_, that "$f(n)$ is proportional to $n$" or "$f(n)$ scales linearly with $n$."

> In formal computer science terms, we define $f(n) = O(n)$ to mean that there exists some constant $c$ **independent** of $n$ such that $f(n) \leq c \cdot n$ as $n$ goes to infinity. So, this notation actually means that $f(n)$ is some asymptotic function $n$ _in the worst case_.

In [ ]:
def decomp__v0(d, n=None):
    # Input is of length `m`
    # Output is of length `n`
    # If possible, we'd like a method that scales like `O(m + n)`

    if n is None:
        n = max(d['inds']) + 1

    # Inputs: `d` - let the size be `m`
    # Outputs: x - let the size be `n`
    # => A good algorithm should scale like `O(m + n)`
    x = [] # O(1)
    for i in range(n): # executes `n` times; O(m * n)
        x.append(decomp_i(i, d)) # build element x[i] => O(m)
    # Loop above scales like O(@???@) rather than O(@???@) <== "bad"
    return x # O(1)

x = decomp__v0(d)
print(x)
assert x == x_true

[1.0, 7.0, 0, 11.0, 0, 6.0, 0, 3.0]


The problem with the above is that it reads `d` for _every_ output element. If the input vector `d` has $m$ nonzeros and the output vector is of length $n$, this procedure does $O(n \cdot m)$ operations instead of the more desirable linear cost of $O(m + n)$ operations.

Can we accomplish the same thing touching each input only once (or only a few times)? Yes!

Consider any function that consumes `m` inputs, executes `k` steps, and returns an output of size `n`. Assume `k` is a constant number of steps that does not depend on `m` or `n`.

```python
def foo(...):    # Input of size `m`
    step_0(...)  # O(m + n)
    step_1(...)  # O(m + n)
    ...
    step_k(...)  # O(m + n)
    return ...   # Output of size `n`
```

The cost is $O(m + n)$ as long as `k` does not depend on `m` or `n`.

In [ ]:
def decomp__v1(d, n=None): # |d| = m elements
    if n is None:
        n = max(d['inds']) + 1 # `max` scales like `O(m)`

    ### YOUR CODE HERE ###
    # 1. Create an empty (all zeros) output of length n # O(n)
    # 2. For each element of d, add the value to the corresponding output element # O(m)
    x = [0] * n # O(n)
    for j, v in zip(d['inds'], d['vals']): # executes `m` times ==> O(m)
        x[j] += v
    return x

y = decomp__v1(d)
print(y)
assert y == x_true

[1.0, 7.0, 0, 11.0, 0, 6.0, 0, 3.0]


## Follow-up exercise: Sparse dot products ##

Consider a dense dot product where the vector elements (or components) include zeros:

In [ ]:
x = [17, 0, 0, -20, -20, 14, 0, 0, -6, 0]
y = [-6, 0, 13, 0, 2, 0, 0, 7, -2, 0]

def dot(x, y):
    return sum(ex*ey for ex, ey in zip(x, y))

dot(x, y)

Here is a variation on this code designed to resemble more closely the mathematical formula for a dot product, which iterates over an index:

$$\mbox{dot}(x, y) = \sum_{i=0}^{n-1} x_i y_i.$$

In [ ]:
def dot__v2(x, y):
    assert len(x) == len(y)
    return sum(x[i]*y[i] for i in range(len(x)))

dot__v2(x, y)

Suppose we want to exploit the zeros by avoiding storing and operating on them. Here are two sparse vectors in the style of `nb1.1.3`:

In [ ]:
dx = {'inds': [0, 3, 4, 5, 8], 'vals': [17, -20, -20, 14, -6]} # O(m1)
dy = {'inds': [0, 2, 4, 7, 8], 'vals': [-6, 13, 2, 7, -2]} # O(m2)
# Hoping for O(m1 + m2)

**Question:** What does a dot product look like for this sparse representation? To simplify the problem a little, let's assume positions are unique.

The terms of the dot product (i.e., $x_i y_i$) are nonzero only when _both_ $x_i \neq 0$ and $y_i \neq 0$. We can detect the common $i$ indices between the two sparse vectors by converting the indices to sets and then finding their intersection:

In [ ]:
def common_inds(dx, dy):
    ### YOUR CODE HERE
    pass

print('Ix =', dx['inds'])
print('Iy =', dy['inds'])
print('common_inds(Ix, Iy):', common_inds(dx, dy))

From this building block, here's a first cut at a dot product "derived" from `dot__v2`:

In [ ]:
# First, recall how to find a value in a `list`
dx['inds'].index(5) # The value `5` is at position 3 of the list `dx['inds']`

In [ ]:
def sp_dot__v0(dx, dy):
    dot = 0
    for i in common_inds(dx, dy):    # O(@???@)
        loc_x = dx['inds'].index(i)  # O(@???@)
        ### YOUR CODE HERE ###
    return dot

sp_dot__v0(dx, dy)

If each vector has an average of $m$ nonzeros, then the cost of this procedure is $O(m^2)$, since there is a hidden $O(m)$ cost to search the list to find the index location.

**Reducing the cost.** To reduce the cost of search, we can ask whether there is some preprocessing that might help. Indeed, in a single pass over the sparse vector, we can construct a "lookup table" that maps indices to values.

In [ ]:
def gen_lookup(d): # `d` is a sparse vector with unique indices
    ### YOUR CODE HERE ###
    pass

print("dx =", dx)
print("lookup table:", gen_lookup(dx))

> **Exercise.** This approach only works when the sparse vector's indices are unique. Can you come up with an alternative method that works when there are nonunique indices?

Combining these two building blocks, here is a more efficient sparse dot:

In [ ]:
def sp_dot__v1(dx, dy):
    tx = gen_lookup(dx)
    ty = gen_lookup(dy)
    I = set(tx.keys()) & set(ty.keys()) # common indices
    return sum(tx[i]*ty[i] for i in I)

sp_dot__v1(dx, dy)

**Benchmarking.** Let's compare the speed of these two sparse dot product implementations against the dense case. Let's start with a small example.

In [ ]:
def decomp(d, n_max=None):
    if n_max is None:
        n_max = (max(d['inds']) if d['inds'] else 0) + 1
    y = [0] * n_max
    for i, v in zip(d['inds'], d['vals']):
        y[i] += v
    return y

print("* Dense case...")
x = decomp(dx)
y = decomp(dy)
%timeit dot(x, y) # Could also try `dot__v2`

print("\n* Sparse dot, version 0...")
%timeit sp_dot__v0(dx, dy)

print("\n* Sparse dot, version 1...")
%timeit sp_dot__v1(dx, dy)

On small inputs, you can see the exact _opposite_ of what you might expect! That's because conversion to sets and construction or manipulation of lookup tables carries an overhead that outweights the benefits of sparse storage when the vectors are small.

Instead, let's try large inputs. Here is a helper function to generate a large random sparse input vector:

In [ ]:
def gen_rand_sparse_vector(n=10, nnz=10, max_val=20, unique=True):
    from random import randint, sample, random, shuffle
    nnz_unique = nnz
    vals_unique = [(i, randint(-max_val, max_val)) for i in sample(range(n), nnz)]
    d_true = {'inds': [i for i, _ in sorted(vals_unique)],
              'vals': [v for _, v in sorted(vals_unique)]}
    if unique:
        return d_true

    vals = []
    for i, v in vals_unique:
        if random() < 0.2: # Duplicate!
            n_dups = randint(2, 5)
            vals += [(i, w) for w in partition_value(v, n_dups)]
        else:
            vals += [(i, v)]
    shuffle(vals)
    d_input = {'inds': [i for i, _ in vals],
               'vals': [v for _, v in vals]}
    return d_input

Let's use `gen_rand_sparse_vector` to generate random sparse input vectors of length 100,000 elements with 1,000 nonzeros each.

In [ ]:
N = 100_000
dx_big = gen_rand_sparse_vector(n=N, nnz=1_000)
dy_big = gen_rand_sparse_vector(n=N, nnz=1_000)

print(f"* dx_big: {len(dx_big['inds']):,} nonzeros.")
print(f"* dy_big: {len(dx_big['inds']):,} nonzeros.")
print(f"* common indices: {len(common_inds(dx_big, dy_big)):,} nonzeros.")

Let's try first converting to dense vectors:

In [ ]:
x_big = decomp(dx_big, n_max=N)
y_big = decomp(dy_big, n_max=N)
print(dot(x_big, y_big)) # note the "correct" answer
%timeit dot(x_big, y_big)

Now let's try our two sparse methods:

In [ ]:
print(sp_dot__v0(dx_big, dy_big)) # check answer against the dense case
%timeit sp_dot__v0(dx_big, dy_big)

print(sp_dot__v1(dx_big, dy_big)) # check answer
%timeit sp_dot__v1(dx_big, dy_big)

You should observe these to be much faster now.

## Summary ##

1. From nb1.1.3, remember to strive for linear time methods.
2. In the sparse dot product problem, using the right data structures helped to achieve efficient code, at least for large-enough inputs.

Next time: Review the basic Python data structures and their tradeoffs.